In [3]:
import numpy as np

from hamming import HammingCode, BitError
from models import BinaryMessage
from transmission import (
    str2bit,
    bit2str,
    pad_last_chunk,
    list_to_chunks,
    chunks_to_list,
    generate_single_bit_error,
    generate_multi_bit_error
)

# Hamming's Code

As part of this question, you are going to use linear error-correcting codes invented by Richard W. Hamming in 1950 to detect errors [1, p.211], [2]. Hamming invented these linear error-correcting codes to detect up to two-bit errors or one-bit errors without detection of uncorrected errors [3]. The linear error-correcting code that encodes four bits of data into seven bits by adding three parity bits. Hamming’s (7,4) algorithm [3] can either correct any single-bit error, or detect all single-bit and two-bit errors as further described in [3]. Error-correcting codes are widely adopted in many kinds of transmission (including WiFi, cell phones, communication with satellites and spacecraft, and digital television) and storage (RAM, disk drives, flash memory, CDs, etc.) [1, p.211].

Hamming discovered a code in which a four-bit message is transformed into a seven-bit codeword. The generator matrix (G), parity-check matrix (H) discovered by Hamming is shown in fig. 2 and the Hamming’s Decoder Matrix (R) as shown in fig. 3. An encoding of a 4-bit binary value (word) \( w \) is a 7-bit vector i.e. the codeword resulting from a matrix-vector product $ c_w = G * w $ [3].

1. Write a simple Hamming encoder program in Python, which, when given a 4-bit binary value, returns the resulting 7-bit binary vector codeword. Also implement the parity check functionality to see if there are any errors, that is to check whether $ H * c_w = \vec{0} $ holds, where \( \vec{0} \) is zero vector.


In [71]:
h = HammingCode(message_bits=4, mode="detect")  # 4 bit message

message = np.array([1, 0, 1, 1])
codeword = h.encode(message)

print(message)
print(codeword)

# check for errors and correct them if possible‚:
codeword = h.check_correct(codeword)
print(codeword)

[1 0 1 1]
[1 0 1 1 0 1 0]
[1 0 1 1 0 1 0]


2. Create a decoder program in Python, which, when given a 7-bit vector codeword, returns the original 4-bit vector word. That is, if we are given a 4-bit word \( w \), and we apply our encoder to return a codeword $ c_w = G * w $, and then we apply the decoder matrix (R) (fig. 3) to $ c_w $, then it should return the original word, such that $ R * c_w = w $.

In [72]:
message_recovered = h.decode(codeword)
print(message_recovered)

assert np.array_equal(message, message_recovered)

[1 0 1 1]


3. Test your code by creating a few 4-bit vectors and running encode and then decode to check if you end up with the original 4-bit vector. Also, test your code with some errors and see if the parity check can identify the errors if so, to what extent.

In [81]:
# 1-bit errors can be detected and corrected
h = HammingCode(message_bits=4, mode="correct")
n_tests = 5

for i in range(n_tests):
    # generate random message
    message = np.random.randint(0, 2, 4)
    codeword = h.encode(message)
    print("Message:     ", message)
    print("Codeword:    ", codeword)

    # similuate transmission errors: only one bit is flipped
    codeword_transmitted = h.simulate_1bit_flip(codeword)
    print("Transmitted: ", codeword_transmitted)

    # check for errors and correct them if possible
    codeword_checked = h.check_correct(codeword_transmitted)
    message_recovered = h.decode(codeword_checked)
    print("Recovered:   ", message_recovered)
    print("_" * 36)
    assert np.array_equal(message, message_recovered)

Message:      [1 1 0 1]
Codeword:     [1 1 0 1 0 0 1]
Transmitted:  [1 1 1 1 0 0 1]
Recovered:    [1 1 0 1]
____________________________________
Message:      [0 1 0 1]
Codeword:     [0 1 0 1 0 1 0]
Transmitted:  [0 1 1 1 0 1 0]
Recovered:    [0 1 0 1]
____________________________________
Message:      [0 0 0 0]
Codeword:     [0 0 0 0 0 0 0]
Transmitted:  [0 0 0 0 1 0 0]
Recovered:    [0 0 0 0]
____________________________________
Message:      [0 1 1 1]
Codeword:     [0 1 1 1 1 0 0]
Transmitted:  [0 1 1 0 1 0 0]
Recovered:    [0 1 1 1]
____________________________________
Message:      [1 1 1 1]
Codeword:     [1 1 1 1 1 1 1]
Transmitted:  [0 1 1 1 1 1 1]
Recovered:    [1 1 1 1]
____________________________________


In [7]:
# 1- or 2-bit errors can be detected, but NOT corrected
h = HammingCode(message_bits=4, mode="detect")  # <- mode changed
n_tests = 5

for i in range(n_tests):
    # generate random message
    message = np.random.randint(0, 2, 4)
    codeword = h.encode(message)
    print("Message:     ", message)
    print("Codeword:    ", codeword)

    # similuate transmission errors: 0, 1 or 2 bits are flipped
    n_errors = np.random.randint(0, 3)
    if n_errors == 1:
        codeword_transmitted = h.simulate_1bit_flip(codeword)
    elif n_errors == 2:
        codeword_transmitted = h.simulate_2bit_flip(codeword)
    else:
        codeword_transmitted = codeword
    print("Transmitted: ", codeword_transmitted)

    # check for errors. If errors are detected, an exception is raised
    print(f"Number of errors: {n_errors}")
    try:
        codeword_checked = h.check_correct(codeword_transmitted)
        print("No errors detected.")
    except BitError as e:
        print(f"Error detected: {e}")
    print("_" * 36)

Message:      [0 1 0 0]
Codeword:     [0 1 0 0 1 0 1]
Transmitted:  [0 1 0 0 1 0 1]
Number of errors: 0
No errors detected.
____________________________________
Message:      [1 1 0 1]
Codeword:     [1 1 0 1 0 0 1]
Transmitted:  [1 1 1 1 1 0 1]
Number of errors: 2
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [0 1 0 1]
Codeword:     [0 1 0 1 0 1 0]
Transmitted:  [0 1 0 1 0 0 1]
Number of errors: 2
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [1 1 0 0]
Codeword:     [1 1 0 0 1 1 0]
Transmitted:  [1 1 0 1 1 1 0]
Number of errors: 1
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [0 1 1 1]
Codeword:     [0 1 1 1 1 0 0]
Transmitted:  [0 1 1 1 1 0 0]
Number of errors: 0
No errors detected.
____________________________________


# Use-Case: Transmission over Noisy Channel

### Assumption: only 1-bit errors possible

In [94]:
# Set a message to transmit.
message = "Hello World! I will transmit this message now. I hope it will arrive correctly."

# Activate hamming protection or not. If activated, the message will be encoded and
# corrected using hamming codes. If deactivated, the message will be transmitted as is
# and will likely contain errors.
hamming_protection = False

# Set the length of the packages. The message will be split into packages of this length.
# A usual length is 8. A shorter length protects the message to a higher degree, but also
# increases the overhead.
message_length = 8

# Set the probability of a bit flip. If hamming protection is activated, the probability
# is not relevant, since the hamming code can correct single bit flips.
# Note: we assume that there is no more than one bit flip per package.
error_probability = 0.1

In [161]:
print("Message:      " + message)
message_encoded = str2bit(message)
binary_list, padding_needed = pad_last_chunk(message_encoded, message_length)
message_binary = list_to_chunks(binary_list, message_length)

if hamming_protection:
    h = HammingCode(message_length, mode="correct")
    message_binary_encoded = []
    for row in message_binary:
        message_binary_encoded.append(h.encode(np.array(row)))
    message_binary = np.array(message_binary_encoded)

# simulate noise
error = generate_single_bit_error(BinaryMessage(message_binary), error_probability)
packages_transmitted = (message_binary + error) % 2
nb_errors = np.count_nonzero(error)
print("Nb Errors:   ", nb_errors)

if hamming_protection:
    message_binary_decoded = []
    for row in packages_transmitted:
        corrected = h.check_correct(np.array(row))
        message_binary_decoded.append(h.decode(corrected))
    packages_transmitted = np.array(message_binary_decoded)
message_flat = chunks_to_list(packages_transmitted)
message_flat = message_flat[:(None if padding_needed == 0 else -padding_needed)]

print("Transmitted: ", bit2str(message_flat))

assert message == bit2str(message_flat), "Message was not transmitted correctly!"

Message:      Hello World! I will transmit this message now. I hope it will arrive correctly.
Nb Errors:    2
Transmitted:  Hello World! I will transmit this message now. I hope it will arrive correctly.


### Multi bit errors

In [164]:
# Set a message to transmit.
message = "Hello World! I will transmit this message now. I hope it will arrive correctly."

# Activate hamming protection or not.
hamming_protection = True

# Set the length of the packages
message_length = 4

# Set the probability of a bit flip
error_probability = 0.01

# Now we dont assume that there is only one bit flip per package any more. Depending on
# how probable errors are, we need to decide if we want to correct errors or only detect
# them. If we want to correct errors, we might get incorrect messages, because we cannot
# detect more than 1 bit errors. If we only want to detect errors, we can detect, but not
# correct, up to 2-bit errors per package. Set mode to "correct" or "detect" accordingly.
h = HammingCode(message_length, mode="correct")


# NOTE: in the above cenario, a good solution is reducing the package size. Using a
# package size of 4, the probability of 2-bit errors per package is much lower. Using the
# mode 'correct' in this case means that we will be able to transmit a correct message in
# most cases. It will not promise that the message will be transmitted correctly, but it
# is more likely. Just try it out :)

# message_length = 4
# h = HammingCode(message_length, mode="correct")

In [171]:
print("Message:      " + message)
message_encoded = str2bit(message)
binary_list, padding_needed = pad_last_chunk(message_encoded, message_length)
message_binary = list_to_chunks(binary_list, message_length)

if hamming_protection:
    message_binary_encoded = []
    for row in message_binary:
        message_binary_encoded.append(h.encode(np.array(row)))
    message_binary = np.array(message_binary_encoded)

# simulate noise
error = generate_multi_bit_error(BinaryMessage(message_binary), error_probability)
packages_transmitted = (message_binary + error) % 2
nb_errors = np.count_nonzero(error)
print("Nb Errors:   ", nb_errors)

if hamming_protection:
    message_binary_decoded = []
    for row in packages_transmitted:
        try:
            corrected = h.check_correct(np.array(row))
        except BitError as e:
            print(f"Error detected: {e}")
            corrected = row
        message_binary_decoded.append(h.decode(corrected))
    packages_transmitted = np.array(message_binary_decoded)
message_flat = chunks_to_list(packages_transmitted)
message_flat = message_flat[:(None if padding_needed == 0 else -padding_needed)]

print("Transmitted: ", bit2str(message_flat))

assert message == bit2str(message_flat), "Message was not transmitted correctly!"

Message:      Hello World! I will transmit this message now. I hope it will arrive correctly.
Nb Errors:    13
Transmitted:  Hello World! I will transmit this message now. I hope it will arrive correctly.
